In [1]:
# pip install pyspark

# spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ConfigurationExample") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
schemas = "id long,home_team_id string,away_team_id string,status string,home_team_score long,away_team_score long,live_home_team_score long,live_away_team_score long,home_team_spread float,away_team_spread float,point_total float,date_id long,kickoff_time date,sport_id string"
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schemas) \
    .option("inferSchema", "True") \
    .option("mode", "dropmalformed") \
    .load("./db/match.csv")
df.show()

24/06/18 08:05:38 WARN Utils: Your hostname, bbswsport resolves to a loopback address: 127.0.1.1; using 192.168.1.39 instead (on interface eno1)
24/06/18 08:05:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/18 08:05:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/18 08:05:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-----+------------+------------+------+---------------+---------------+--------------------+--------------------+----------------+----------------+-----------+-------+------------+--------+
|   id|home_team_id|away_team_id|status|home_team_score|away_team_score|live_home_team_score|live_away_team_score|home_team_spread|away_team_spread|point_total|date_id|kickoff_time|sport_id|
+-----+------------+------------+------+---------------+---------------+--------------------+--------------------+----------------+----------------+-----------+-------+------------+--------+
| 7587|          SD|         LAD| Final|              2|              5|                   2|                   5|             1.5|            -1.5|        8.5|    230|  2024-03-20|     mlb|
| 7588|         LAD|          SD| Final|             11|             15|                  11|                  15|            -1.5|             1.5|        8.5|    231|  2024-03-21|     mlb|
| 7590|         BAL|         LAA| Final|     

# Cout row data

In [3]:
rdd = spark.sparkContext.textFile('./db/match.csv')
rdd.count()

2431

# read 20 row data

In [4]:
# không lấy row 1 header
rdd = rdd.filter(lambda x: x.split(',')[0] != 'match_id')
rdd.take(20)

['id,home_team_id,away_team_id,status,home_team_score,away_team_score,live_home_team_score,live_away_team_score,home_team_spread,away_team_spread,point_total,date_id,kickoff_time,sport_id',
 '7587,SD,LAD,Final,2,5,2,5,1.5,-1.5,8.5,230,2024-03-20,mlb',
 '7588,LAD,SD,Final,11,15,11,15,-1.5,1.5,8.5,231,2024-03-21,mlb',
 '7590,BAL,LAA,Final,11,3,11,3,-1.5,1.5,8,232,2024-03-28,mlb',
 '7592,CHW,DET,Final,0,1,0,1,1.5,-1.5,7.5,232,2024-03-28,mlb',
 '7593,KC,MIN,Final,1,4,1,4,1.5,-1.5,8.5,232,2024-03-28,mlb',
 '7594,HOU,NYY,Final,4,5,4,5,-1.5,1.5,9,232,2024-03-28,mlb',
 '7595,MIA,PIT,Final,5,6,5,6,-1.5,1.5,7.5,232,2024-03-28,mlb',
 '7596,SD,SF,Final,6,4,6,4,1.5,-1.5,7.5,232,2024-03-28,mlb',
 '7597,LAD,STL,Final,7,1,7,1,-1.5,1.5,8.5,232,2024-03-28,mlb',
 '7598,TB,TOR,Final,2,8,2,8,-1.5,1.5,7.5,232,2024-03-28,mlb',
 '7599,CIN,WSH,Final,8,2,8,2,-1.5,1.5,9.5,232,2024-03-28,mlb',
 '7600,TEX,CHC,Final,4,3,4,3,-1.5,1.5,8.5,232,2024-03-28,mlb',
 '7601,OAK,CLE,Final,0,8,0,8,1.5,-1.5,7.5,232,2024-03-29,m

# đếm số lượng những trận đấu của từng đội

In [5]:
mapped_hometeam_rdd = rdd.map(lambda x: (x.split(',')[1], 1))
mapped_awayteam_rdd = rdd.map(lambda x: (x.split(',')[2], 1))
mapped_awayteam_rdd.take(10)

[('away_team_id', 1),
 ('LAD', 1),
 ('SD', 1),
 ('LAA', 1),
 ('DET', 1),
 ('MIN', 1),
 ('NYY', 1),
 ('PIT', 1),
 ('SF', 1),
 ('STL', 1)]

In [6]:
reduced_hometeam_rdd = mapped_hometeam_rdd.reduceByKey(lambda x, y: x + y)
reduced_awayteam_rdd = mapped_awayteam_rdd.reduceByKey(lambda x, y: x + y)
reduced_awayteam_rdd.collect()


[('away_team_id', 1),
 ('LAD', 81),
 ('LAA', 81),
 ('NYY', 81),
 ('PIT', 81),
 ('WSH', 81),
 ('CLE', 81),
 ('BOS', 81),
 ('COL', 81),
 ('CIN', 81),
 ('MIA', 81),
 ('OAK', 81),
 ('TB', 81),
 ('BAL', 81),
 ('ARI', 81),
 ('SD', 81),
 ('DET', 81),
 ('MIN', 81),
 ('SF', 81),
 ('STL', 81),
 ('TOR', 81),
 ('CHC', 81),
 ('MIL', 81),
 ('ATL', 81),
 ('KC', 81),
 ('TEX', 81),
 ('CHW', 81),
 ('NYM', 81),
 ('PHI', 81),
 ('HOU', 81),
 ('SEA', 81)]

In [7]:
reduced_hometeam_rdd.collect()

[('LAD', 81),
 ('BAL', 81),
 ('MIA', 81),
 ('TB', 81),
 ('CIN', 81),
 ('OAK', 81),
 ('ARI', 81),
 ('WSH', 81),
 ('NYY', 81),
 ('COL', 81),
 ('PIT', 81),
 ('LAA', 81),
 ('CLE', 81),
 ('BOS', 81),
 ('home_team_id', 1),
 ('SD', 81),
 ('CHW', 81),
 ('KC', 81),
 ('HOU', 81),
 ('TEX', 81),
 ('SEA', 81),
 ('NYM', 81),
 ('PHI', 81),
 ('CHC', 81),
 ('MIL', 81),
 ('MIN', 81),
 ('STL', 81),
 ('DET', 81),
 ('SF', 81),
 ('ATL', 81),
 ('TOR', 81)]

# đếm số lượng đội thắng 

In [8]:
def is_valid_row(row):
    try:
        parts = row.split(',')
        match_id = parts[0]
        home_team = parts[1]
        away_team = parts[2]
        home_score = int(parts[4])
        away_score = int(parts[5])
        return True
    except Exception as e:
        print(f"Invalid row: {row}, Error: {e}")
        return False

rdd = rdd.filter(is_valid_row)

In [9]:
mapped_win_rdd = rdd.map(lambda x: (x.split(',')[1], 1) if int(x.split(',')[4]) >= int(x.split(',')[5]) else (x.split(',')[2], 1))

In [10]:

reduced_win_rdd = mapped_win_rdd.reduceByKey(lambda x, y: x + y)
reduced_win_rdd.collect()

Invalid row: 8708,OAK,KC,Scheduled,,,,,,,,315,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8709,WSH,ARI,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8710,CLE,SEA,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8711,MIN,TB,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8712,CHW,HOU,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8713,COL,LAD,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8714,OAK,KC,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8715,NYY,BAL,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8716,STL,SF,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with

[('LAD', 42),
 ('BAL', 45),
 ('NYY', 50),
 ('PIT', 33),
 ('CIN', 34),
 ('CLE', 44),
 ('BOS', 35),
 ('ARI', 33),
 ('TB', 33),
 ('WSH', 33),
 ('COL', 24),
 ('LAA', 26),
 ('OAK', 26),
 ('MIA', 23),
 ('SD', 37),
 ('DET', 33),
 ('MIN', 38),
 ('TOR', 33),
 ('TEX', 33),
 ('MIL', 40),
 ('ATL', 37),
 ('SEA', 40),
 ('SF', 34),
 ('STL', 33),
 ('PHI', 47),
 ('KC', 40),
 ('CHC', 33),
 ('HOU', 32),
 ('CHW', 18),
 ('NYM', 31)]

In [11]:
reduced_win_rdd.sortBy(lambda x: x[1], ascending=False).collect()[0]

('NYY', 50)

In [12]:
rdd.take(10)

Invalid row: id,home_team_id,away_team_id,status,home_team_score,away_team_score,live_home_team_score,live_away_team_score,home_team_spread,away_team_spread,point_total,date_id,kickoff_time,sport_id, Error: invalid literal for int() with base 10: 'home_team_score'


['7587,SD,LAD,Final,2,5,2,5,1.5,-1.5,8.5,230,2024-03-20,mlb',
 '7588,LAD,SD,Final,11,15,11,15,-1.5,1.5,8.5,231,2024-03-21,mlb',
 '7590,BAL,LAA,Final,11,3,11,3,-1.5,1.5,8,232,2024-03-28,mlb',
 '7592,CHW,DET,Final,0,1,0,1,1.5,-1.5,7.5,232,2024-03-28,mlb',
 '7593,KC,MIN,Final,1,4,1,4,1.5,-1.5,8.5,232,2024-03-28,mlb',
 '7594,HOU,NYY,Final,4,5,4,5,-1.5,1.5,9,232,2024-03-28,mlb',
 '7595,MIA,PIT,Final,5,6,5,6,-1.5,1.5,7.5,232,2024-03-28,mlb',
 '7596,SD,SF,Final,6,4,6,4,1.5,-1.5,7.5,232,2024-03-28,mlb',
 '7597,LAD,STL,Final,7,1,7,1,-1.5,1.5,8.5,232,2024-03-28,mlb',
 '7598,TB,TOR,Final,2,8,2,8,-1.5,1.5,7.5,232,2024-03-28,mlb']

In [13]:
mapped_date_rdd = rdd.map(lambda x: (x.split(',')[-2]))
mapped_date_rdd.min()

Invalid row: 8708,OAK,KC,Scheduled,,,,,,,,315,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8709,WSH,ARI,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8710,CLE,SEA,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8711,MIN,TB,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8712,CHW,HOU,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8713,COL,LAD,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8714,OAK,KC,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8715,NYY,BAL,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with base 10: ''
Invalid row: 8716,STL,SF,Scheduled,,,,,,,,316,2024-06-20,mlb, Error: invalid literal for int() with

'2024-03-20'